In [9]:
import jax
from jax import numpy as jnp, random
import numpy as np
import pandas as pd
from mondeq.modules import MonLinear, Relu, fcmon
from mondeq.splittings import build_peacemanrachford_update
from utils import commutation_matrix_sp, vec
from functools import partial
import pandas as pd

In [16]:
dyn = partial(build_peacemanrachford_update, 1.)
def stats_TK(d, k, Z0, X, key):
    skey = random.split(key)
    lin_module = MonLinear(d, k, key=skey[0])
    nonlin_module = Relu()
    u = random.normal(skey[1], shape=(k,))
    Ju = jax.jacrev(lambda u: fcmon(dyn, max_iter, tol, u, nonlin_module, lin_module, Z0, X))(u)
    Jlin = jax.jacrev(lambda lin_module: fcmon(dyn, max_iter, tol, u, nonlin_module, lin_module, Z0, X), allow_int=True)(lin_module)
    
    H1_A = vprod(Jlin.p_A)
    H1_B = vprod(Jlin.p_B)

    H1 = H1_A + H1_B
    H2 = vprod(Jlin.p_U)
    H3 = Ju @ Ju.T
    H = H1 + H2 + H3
    return {
        "eigmin_H1_A": jnp.linalg.eigvalsh(H1_A)[0],
        "eigmin_H1_B": jnp.linalg.eigvalsh(H1_B)[0],

        "eigmin_H1": jnp.linalg.eigvalsh(H1)[0],
        "eigmin_H2": jnp.linalg.eigvalsh(H2)[0],
        "eigmin_H3": jnp.linalg.eigvalsh(H3)[0],

        "eigmin_H": jnp.linalg.eigvalsh(H)[0],
    }
stats_TK_jit = jax.jit(stats_TK, static_argnums=(0, 1))

In [11]:
num_runs = 100
num_samples = 10
d = 3
k = 10
key = random.PRNGKey(0)
max_iter = 40
tol = 1e-5
def vprod(M):
    return jnp.einsum("ihk,jhk->ij", M, M)

In [17]:
Z0 = jnp.zeros(shape=(num_samples, k))
row_list = []
for i in range(num_runs):
    key, skey = random.split(key)
    X = np.random.normal(size=(num_samples, d))
    result = stats_TK_jit(d, k, Z0, X, skey)
    row_list.append(result)
df = pd.DataFrame(row_list)

In [18]:
df

,eigmin_H,eigmin_H1,eigmin_H1_A,eigmin_H1_B,eigmin_H2,eigmin_H3
0,0.76547813,0.48315856,0.21519268,0.21311027,0.1313556,7.0169466e-05
1,0.30839333,0.045581773,0.033340856,3.5762494e-06,0.23173563,0.00011562902
2,0.00010101465,1.0281336e-05,3.933892e-06,6.79349e-07,3.344173e-05,-1.269172e-07
3,0.07540217,0.032035477,0.015126066,0.011308913,0.0077168224,2.654346e-05
4,0.03356142,0.016688617,0.00529133,0.00973686,0.009901461,-2.8717366e-07
...,...,...,...,...,...,...
95,0.0009111568,5.5707642e-05,3.242965e-05,1.9164865e-05,0.0006355348,8.0647524e-08
96,0.0638301,0.016229989,0.0053977547,0.008902158,0.019801708,-1.8638307e-08
97,0.15727855,0.07894204,0.032795854,0.026625708,0.021980008,-1.4032429e-07
98,0.109212115,0.0322613,0.0070498036,0.017665314,0.03930931,1.7714196e-06
